<a href="https://colab.research.google.com/github/yuthelastleaf/APTTechnology/blob/main/ToonCrafter_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd /content
!git clone -b dev https://github.com/camenduru/ToonCrafter
%cd /content/ToonCrafter

!apt -y install -qq aria2

# --- 1. 下载全精度原版模型 (High Quality) ---
# 既然显存够用，我们直接下载原版 model.ckpt，不带 _fp16 后缀
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/ToonCrafter/resolve/main/model.ckpt -d /content/ToonCrafter/checkpoints/tooncrafter_512_interp_v1 -o model.ckpt
# ----------------------------------------

# --- 2. 【必须执行】修复环境依赖报错 ---
# 这些修复是针对 Python 环境的，跟显卡没关系，必须保留
!sed -i 's/numpy==1.24.2/numpy/g' requirements.txt
!sed -i 's/decord==0.6.0/decord/g' requirements.txt
!sed -i 's/imageio==2.9.0/imageio/g' requirements.txt
!sed -i 's/PyYAML==6.0/PyYAML/g' requirements.txt
!sed -i 's/Pillow==9.5.0/Pillow/g' requirements.txt
!sed -i 's/pandas==2.0.0/pandas/g' requirements.txt

# --- 3. 手动安装关键库 ---
# 提前安装 pytorch_lightning 避免安装脚本因为其他包失败而跳过它
!pip install pytorch_lightning omegaconf open_clip_torch
!pip install decord

# --- 4. 安装剩余依赖 ---
!pip install -r requirements.txt
!pip install einops -U
!pip uninstall keras -y

# --- 5. 启动 ---
print("正在启动高性能版 ToonCrafter...")
!python gradio_app.py

/content
fatal: destination path 'ToonCrafter' already exists and is not an empty directory.
/content/ToonCrafter
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
6345a4|OK  |   238MiB/s|/content/ToonCrafter/checkpoints/tooncrafter_512_interp_v1/model.ckpt

Status Legend:
(OK):download completed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 5.5 MB/s eta 0:00:00
  Using cached einops-0.3.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached omegaconf-2.1.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached pytorch_lightning-1.9.3-py3-none-any.whl.me

In [3]:
import os
import sys

# 1. 确保在正确的目录下
if os.path.exists("/content/ToonCrafter"):
    os.chdir("/content/ToonCrafter")

print("正在修复依赖环境...")

# 2. 安装核心依赖 (加上 xformers 防止代码报错)
os.system("pip install xformers")  # 【新增】解决 NameError 关键
os.system("pip install kornia")

# 3. 修改 requirements.txt 防止 PyTorch 版本冲突
req_file = "requirements.txt"
if os.path.exists(req_file):
    with open(req_file, "r") as f:
        lines = f.readlines()
    with open(req_file, "w") as f:
        for line in lines:
            # 过滤掉强制指定版本的 torch/torchvision，直接用系统自带的
            if "torch" in line or "torchvision" in line:
                continue
            f.write(line)
    print("requirements.txt 已修正 (移除旧版 torch 限制)")

# 4. 安装剩余的兼容依赖
os.system("pip install -r requirements.txt")

# 5. 解决 OpenCLIP 版本报错 (降级到 2.20.0)
print("正在调整 open_clip_torch 版本...")
os.system("pip uninstall -y open_clip_torch")
os.system("pip install open_clip_torch==2.20.0")

# 6. 【保险措施】自动修补代码文件
# 防止即使安装了 xformers 依然因为路径问题报错
target_file = "/content/ToonCrafter/lvdm/models/autoencoder_dualref.py"
if os.path.exists(target_file):
    with open(target_file, "r") as f:
        code = f.read()

    # 简单的字符串替换，给 xformers 调用加上 try-except 保护
    if "xformers.ops.memory_efficient_attention" in code and "try:" not in code:
        print("正在为 autoencoder_dualref.py 应用防崩溃补丁...")
        patch = """
        try:
            import xformers.ops
            out = xformers.ops.memory_efficient_attention(q, k, v)
        except (ImportError, NameError, AttributeError):
            import torch.nn.functional as F
            q_in = q.transpose(1, 2)
            k_in = k.transpose(1, 2)
            v_in = v.transpose(1, 2)
            out = F.scaled_dot_product_attention(q_in, k_in, v_in)
            out = out.transpose(1, 2)
        """
        new_code = code.replace("out = xformers.ops.memory_efficient_attention(q, k, v)", patch)
        with open(target_file, "w") as f:
            f.write(new_code)

print("✅ 环境修复完成！正在启动 ToonCrafter...")

# 7. 启动程序
!python gradio_app.py

正在修复依赖环境...
requirements.txt 已修正 (移除旧版 torch 限制)
✅ 环境修复完成！正在启动 ToonCrafter...
no module 'xformers'. Processing without...
no module 'xformers'. Processing without...
AE working on z of shape (1, 4, 32, 32) = 4096 dimensions.
/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
2025-12-24 05:27:13.067800: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-24 05:27:13.085222: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registe

In [2]:
import os

print("正在修复 open_clip_torch 版本冲突...")

# 1. 卸载导致报错的新版本
os.system("pip uninstall -y open_clip_torch")

# 2. 安装兼容的旧版本 (2.20.0 是已知稳定的版本)
os.system("pip install open_clip_torch==2.20.0")

print("✅ 依赖库已降级。正在重启 ToonCrafter...")

# 3. 再次启动程序
!python gradio_app.py

正在修复 open_clip_torch 版本冲突...
✅ 依赖库已降级。正在重启 ToonCrafter...
python3: can't open file '/content/gradio_app.py': [Errno 2] No such file or directory


In [5]:
import os
import time

# 1. 定义损坏文件的路径
ckpt_path = "/content/ToonCrafter/checkpoints/tooncrafter_512_interp_v1/model.ckpt"
ckpt_dir = os.path.dirname(ckpt_path)

print(f"正在检查文件: {ckpt_path}")

# 2. 删除损坏的文件
if os.path.exists(ckpt_path):
    file_size = os.path.getsize(ckpt_path) / (1024 * 1024) # MB
    print(f"检测到旧文件大小: {file_size:.2f} MB")
    print("正在删除损坏的文件...")
    os.remove(ckpt_path)
else:
    print("文件不存在，准备下载...")

# 确保目录存在
os.makedirs(ckpt_dir, exist_ok=True)

# 3. 重新下载模型 (使用 HuggingFace 源，约 4.6GB)
print("正在开始重新下载 model.ckpt，请耐心等待 (约 4.6GB)...")
# 使用 wget 下载，-O 指定输出文件名
download_cmd = f"wget -c https://huggingface.co/Doubiiu/ToonCrafter/resolve/main/model.ckpt -O {ckpt_path}"
os.system(download_cmd)

# 4. 验证下载是否成功
if os.path.exists(ckpt_path):
    new_size = os.path.getsize(ckpt_path) / (1024 * 1024 * 1024) # GB
    print(f"下载完成！当前文件大小: {new_size:.2f} GB")
    if new_size < 1.0:
        print("⚠️ 警告：文件似乎仍然过小，可能下载失败，请检查网络或重新运行。")
    else:
        print("✅ 文件校验通过。")

        # 5. 再次尝试启动
        print("正在重启 ToonCrafter...")
        os.system("python gradio_app.py")
else:
    print("❌ 下载失败，未找到文件。")

正在检查文件: /content/ToonCrafter/checkpoints/tooncrafter_512_interp_v1/model.ckpt
检测到旧文件大小: 10015.96 MB
正在删除损坏的文件...
正在开始重新下载 model.ckpt，请耐心等待 (约 4.6GB)...
下载完成！当前文件大小: 9.78 GB
✅ 文件校验通过。
正在重启 ToonCrafter...
